# Problem : To classify the input  videoClip as walking or jogging  i.e. Video_Classification 

## Approach: there can be many ways to approach this problem , well considering computational alg. I opt for follwing approach
1.Using squeeznet(pretrained )  to extract feature from image frames in sequence and then feeding it to LSTM to classify the label

# <h1>Libraries and Utilities:</h1>

<h2>Keras with tensorflow as backend</h2>
Description: Deep learning library for Training and implementing deep learning architectures, for this task main training model uses a class of deep models RNN, follwed  by a simple MLP .

<h2>FFMPEG</h2>
for extractions of frames from videos , A specific number of frames has been used limited to 40
so as to avoid redundancy
<h2>Scipy, numpy </h2>
for image/matrix manipulation


In [1]:
import keras

Using TensorFlow backend.


In [ ]:
#First we would like to have frames extracted from the video file , we will use  

In [12]:
import numpy as np

In [7]:
pwd

'D:\\pytorch_learning\\frames'

In [8]:
ls

 Volume in drive D is E:
 Volume Serial Number is 1A44-AA9C

 Directory of D:\pytorch_learning\frames

06/24/2018  12:53 PM    <DIR>          .
06/24/2018  12:53 PM    <DIR>          ..
06/24/2018  12:53 PM    <DIR>          jogging
06/24/2018  12:52 PM    <DIR>          walking
               0 File(s)              0 bytes
               4 Dir(s)  217,410,707,456 bytes free


### So the basic idea should be, 
1. for every video i extract 40 frames using FFMPEG,
2. resize each frame to size >> (128,128,3)
3. pass it through vgg16 or squeeznet (pretrained ) and store them into numpy array.
4. input(128,128,3) <squeeznet> output(512)
5. and hence we will generate the Final_data_array of size>>X>>(m,40,512) #m is total no. of clips
6. now each video clip is an array of size >> (40,512)
7. create labels y binary vector >> labels binary_vector will be of size =m
8. then we will feed this to  our RNN .

In [ ]:
#for every video extract 40 frames using ffmpeg , read all the images , resize them, 
#pass them through cnn , write them into np array

In [22]:
cd int_data/Data/

D:\int_data\Data


In [23]:
pwd

'D:\\int_data\\Data'

In [25]:
import matplotlib.pyplot as plt

In [31]:
from keras.layers import Flatten

In [41]:
from keras_squeezenet import SqueezeNet

In [42]:
model_sq=SqueezeNet(include_top=False,weights="imagenet",input_shape=(128,128,3))

In [43]:
model_sq.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 63, 63, 64)   1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 63, 63, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 31, 31, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [44]:
from keras.layers import GlobalMaxPooling2D

In [45]:
t=GlobalMaxPooling2D()(model_sq.output)

In [46]:
t

<tf.Tensor 'global_max_pooling2d_1/Max:0' shape=(?, 512) dtype=float32>

In [47]:
model_fin=Model([model_sq.input],t)

In [48]:
model_fin.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 63, 63, 64)   1792        input_3[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 63, 63, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 31, 31, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [50]:
import os


### Preparing RNN-feedable data 

In [51]:
jog_files=os.listdir("jogging/")

In [90]:
jogging_final_data=np.zeros(shape=(50,40,512))

In [92]:
for ind,vid in enumerate(jog_files):  #iterating over all  Jogging videoClips
    
    arr=np.zeros(shape=(40,128,128,3))
    vid_file="jogging/"+vid
    
    !ffmpeg  -i $vid_file -vframes 40 frames/jog%02d.jpg   # FFMPEG to extract frames of each clip
    
    for i,img in enumerate(os.listdir("frames/")):
        img=plt.imread("frames/"+img)
        img=imresize(img,(128,128))
        arr[i]=img
        #print(str(i)+"th image")
    ! rm  frames/*
    pred=model_fin.predict(arr)
    jogging_final_data[ind]=pred
    
    print(str(ind)+"th video done")

ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

0th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

1th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

...

46th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

47th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

48th video done


In [93]:
len(jog_files)

49

In [94]:
jogging_final_data.shape #this is our final jogging training data 50 videoClips each have 40 frames

(50, 40, 512)

In [98]:
jogging_final_data[1]

array([[ 0.        ,  0.        ,  9.62548351, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.62548351, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  9.62548351, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        , 36.67172623, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 28.13210678, ...,  0.        ,
        31.21777725,  0.        ],
       [ 0.        ,  0.        , 31.35561562, ...,  0.        ,
        21.76806641,  0.        ]])

### Similary For Walking videoClips

In [99]:
walk_vids=os.listdir("Walking/")

In [101]:
walking_final_data=np.zeros(shape=(50,40,512))

In [102]:
for ind,vid in enumerate(walk_vids):
    
    arr=np.zeros(shape=(40,128,128,3))
    vid_file="Walking//"+vid
    
    !ffmpeg  -i $vid_file -vframes 40 frames/walk%02d.jpg
    
    for i,img in enumerate(os.listdir("frames/")):
        img=plt.imread("frames/"+img)
        img=imresize(img,(128,128))
        arr[i]=img
        #print(str(i)+"th image")
    ! rm  frames/*
    pred=model_fin.predict(arr)
    walking_final_data[ind]=pred
    print(str(ind)+"th video done")

ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

0th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

1th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

...

48th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

49th video done


ffmpeg version N-91330-ga990184007 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 7.3.0 (GCC)
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-bzlib --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth
  libavutil      56. 18.102 / 56. 18.102
  libavcodec     58. 2

IndexError: index 50 is out of bounds for axis 0 with size 50

In [103]:
walking_final_data.shape

(50, 40, 512)

### Sequential_Model Architechture 

In [104]:
from keras.layers import Dense,LSTM,Dropout
from keras.models import Sequential

In [145]:
#this is the main trainable model , we are going to use ,
model = Sequential()
model.add(LSTM(256, return_sequences=False,    #spit out a 1024 dimension vector based on given input sequence,
                                              #input sequence is a 40 frame video 
                       input_shape=(40,512),
                       dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  #since only  one class ,sigmoid is the activation

In [146]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 820,481
Trainable params: 820,481
Non-trainable params: 0
_________________________________________________________________


### Generating a binary_class_label vector (or labels or  y )

In [109]:
cat2index={"walking":0,"jogging":1}

jogging has 49,walking has 50

In [110]:
labels=[0 for i in range(49)]+[1 for i in range(50)]

In [111]:
len(labels)

99

### Compiling 

In [147]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [116]:
jogging_final_data=jogging_final_data[:49]

In [117]:
jogging_final_data.shape

(49, 40, 512)

In [118]:
jogging_final_data.shape

(49, 40, 512)

In [119]:
walking_final_data.shape

(50, 40, 512)

In [122]:
### Concating the Jogging_data_array and Walking_data_array  to create  final_data_array
final_data=np.concatenate((walking_final_data,jogging_final_data),axis=0)

In [123]:
final_data.shape

(99, 40, 512)

### Training

In [166]:
model.fit(final_data,labels,batch_size=8,epochs=10,callbacks=[cb])

Epoch 1/10
99/99 [==============================] - 2s 25ms/step - loss: 0.1462 - acc: 0.9394
Epoch 2/10
99/99 [==============================] - 2s 25ms/step - loss: 0.1601 - acc: 0.9192
Epoch 3/10
99/99 [==============================] - 2s 24ms/step - loss: 0.1625 - acc: 0.9192
Epoch 4/10
99/99 [==============================] - 3s 25ms/step - loss: 0.2075 - acc: 0.9293
Epoch 5/10
99/99 [==============================] - 2s 22ms/step - loss: 0.1737 - acc: 0.9192
Epoch 6/10
99/99 [==============================] - 2s 23ms/step - loss: 0.2675 - acc: 0.9192
Epoch 7/10
99/99 [==============================] - 2s 25ms/step - loss: 0.0783 - acc: 0.9899
Epoch 8/10
99/99 [==============================] - 3s 27ms/step - loss: 0.1258 - acc: 0.9697
Epoch 9/10
99/99 [==============================] - 3s 25ms/step - loss: 0.1148 - acc: 0.9495
Epoch 10/10
99/99 [==============================] - 3s 27ms/step - loss: 0.1578 - acc: 0.9192


This pretrained lite CNN model ie. squeeznet and RNN Duo gives satisfactory result

### Further PlayArounds to increase accuracy 
1. we can increase the no. of frames from 40 to 100 or may be 200 
   which might give better results as we adds more training examples (or frames) 
2. we can use use Bidirectional LSTM instead for bidirectional information flow .
3. we can use another CNN architecture like VGG-16, (provided it'll be more computational)
4. can try another approach may with conv3D

In [ ]:
#saving the model

In [136]:
from keras.models import save_model

In [167]:
save_model(model,"model_1.h5")

In [141]:
from keras.models import load_model

In [142]:
model_loaded=load_model("model_1.h5")

In [143]:
model_loaded.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1024)              6295552   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 6,426,881
Trainable params: 6,426,881
Non-trainable params: 0
_________________________________________________________________
